# Pricing of caplets, floorlets, swaptions with Hull and White 1 factor model

### *Objectives*: Calibrating and using Hull and White 1 factor (also named Gaussian HJM 1 factor, Linear Gauss Markov or Extended Vacisek) model, calculation of vanilla caplets, floorlets and swaptions.

We calculate the values of the instruments using Monte Carlo and partial differential equations (PDE) methods.

1. We define caplets, floorlets and swaptions with their characteristics.   
2. We calibrate the Hull and White 1 factor (HW1F) model.  
3. We implement Monte Carlo in HW1F model.   
4. We calculate the instruments' price using Monte Carlo in HW1F model.  

In [35]:
"""
pricer_HW1F_for_vanilla_caplet_floorlet_swaption.ipynb
======================================================
This script calculates pricing caplets, floorlets and swaptions, using the Hull and White 1 factor model.

Dependencies:
- numpy
- math
- time
- scipy.stats
- scipy.interpolate
- scipy.optimize 
"""

import numpy as np
import math
import time

from scipy.stats import norm
from scipy.interpolate import interp1d
from scipy import optimize
from scipy.linalg import solve_banded

## Calculate the value of european option

We calculate the value of europeam options with the Black formula and the normal formula.  
We tailor the calculation depending on the nature of the priced instrument :    
- if a caplet/floorlet we have a price at a single time.  
- if a swaption we have prices at multiple times.  

### Calculate the value of a call or put option with the Black-Scholes formula

$C_0 = S_0\mathcal{N}(d1) - Ke^{-rT}\mathcal{N}(d2)$

*with*:   
$d1 = \frac{log\frac{S_0}{K}+(r+\frac{\sigma^2}{2})T}{\sigma \sqrt{T}}$    

$d2 = d1 - \sigma \sqrt{T}$


### Calculate the value of a call or put option with the normal formula

$C_0 = (F_0-K)  \mathcal{N}(d) + \sigma_n \sqrt{T} \phi(d)$.  --------- / --------- $P_0 = (K-F_0)  \mathcal{N}(-d) + \sigma_n \sqrt{T} \phi(d)$.  

*with*:   
$d = \frac{F_0 - K}{\sigma_n \sqrt{T}}$   

The formula to calculate the value of an option using the normal model is found from the stochastic formula $F_T = F_0 + \sigma_n W_T$ with $W_T$ the brownian movement.  

In [36]:
"""
Black & normal pricing formulas
"""

# standard version : T, K, F, sigma must be single floats
def bs_call(T, K, F, sigma) :
	if (T==0 or sigma==0) : return max(F-K, 0)
	sigma_sqrt_T = sigma * math.sqrt(T)
	d1 = (math.log(F/K) + 0.5 * sigma**2 * T) / sigma_sqrt_T
	d2 = d1 - sigma_sqrt_T
	return F * norm.cdf(d1) - K * norm.cdf(d2)

# numpy compatible version (slower for non arrays, faster for large arrays)
def bs_call_np(T, K, F, sigma) :
	sigma_sqrt_T = sigma * np.sqrt(T)
	d1 = (np.log(F/K) + 0.5 * sigma**2 * T) / sigma_sqrt_T
	d2 = d1 - sigma_sqrt_T
	return F * norm.cdf(d1) - K * norm.cdf(d2)

# standard version : T, K, F, sigma must be single floats
def bs_put(T, K, F, sigma) :
	return bs_call(T, K, F, sigma) - F + K

# numpy compatible version (slower for non arrays, faster for large arrays)
def bs_put_np(T, K, F, sigma) :
	return bs_call_np(T, K, F, sigma) - F + K

# standard version : T, K, F, sigma must be single floats
def bs_option(call_put, T, K, F, sigma) :
	if (call_put == 'call') : return bs_call(T, K, F, sigma)
	else : return bs_put(T, K, F, sigma)

# numpy compatible version (slower for non arrays, faster for large arrays)
def bs_option_np(call_put, T, K, F, sigma) :
	return np.where (call_put == 'call', bs_call_np(T, K, F, sigma), bs_put_np(T, K, F, sigma))

# standard version : T, K, F, sigma must be single floats
def norm_call(T, K, F, sigma) :
	if (T==0 or sigma==0) : return max(F-K, 0)
	sigma_sqrt_T = sigma * math.sqrt(T)
	d = (F - K) / sigma_sqrt_T
	return sigma_sqrt_T * (d * norm.cdf(d) + norm.pdf(d))

# numpy compatible version (slower for non arrays, faster for large arrays)
def norm_call_np(T, K, F, sigma) :
	sigma_sqrt_T = sigma * np.sqrt(T)
	d = (F - K) / sigma_sqrt_T
	return sigma_sqrt_T * (d * norm.cdf(d) + norm.pdf(d))

# standard version : T, K, F, sigma must be single floats
def norm_put(T, K, F, sigma) :
	return norm_call(T, K, F, sigma) - F + K

# numpy compatible version (slower for non arrays, faster for large arrays)
def norm_put_np(T, K, F, sigma) :
	return norm_call_np(T, K, F, sigma) - F + K

# standard version : T, K, F, sigma must be single floats
def norm_option(call_put, T, K, F, sigma) :
	if (call_put == 'call') : return norm_call(T, K, F, sigma)
	else : return norm_put(T, K, F, sigma)

# numpy compatible version (slower for non arrays, faster for large arrays)
def norm_option_np(call_put, T, K, F, sigma) :
	return np.where (call_put == 'call', norm_call_np(T, K, F, sigma), norm_put_np(T, K, F, sigma))

## Get the zero-coupon rates.

In order to discount the value of the following instruments, we get the zero-coupon rates from an extrapolation of a sample of zero-coupon rates with their maturities.

*As a reminder*: We get zero-coupon rates from market instruments such as IOS swaps. Their calculation is detailed in **pricer_multicurve_for_vanilla_swap_cap_floor_swaption**.

### Calculate discount factors from zero-coupon rates.

Discount factors are calculated using continuous compounding.

$DF(t) = e^{-z_c(t) * t}$

In [37]:
"""
Zero-coupon curve
"""
class zc_curve :

	def __init__(self, maturities, zc_rates):
		self.maturities = maturities
		self.zc_rates = zc_rates
		self.zc_rates_interp = interp1d(maturities, zc_rates, kind='cubic', fill_value="extrapolate")

	def df(self, T):
		return np.exp(-self.zc_rates_interp(T)*T)

## Define the caps, floors and swaptions used for calibration of HW1F model.

### 1. Define instrument's primary parameters.
We define the primary parameters of the instruments:
- type of option : "call" or "put"
- expiry time i.e. fixing date
- start time i.e. effective start of the swap
- pay time i.e. payment date (and end of the swap)
- year fraction i.e period of time
- strike
- nominal

### 2. Define instrument's specialized parameters.
We define parameters depending on the type of instrument:
- as a swaption, payment dates occur once every year during the tenor 
- as a caplet or floorlet, payment date occurs once 

### 3. Calculate needed discounts factors, forward rate, and present value of cap/floor and underlying.
- We calculate the discount factors corresponding to the start dates and payment/end dates for each CF.  

- We calculate the forward rate from the relation $PV_{\text{floating leg}} = PV_{\text{fixed leg}}$ in a swap i.e. $DF(0) - DF(T) = F\sum_{i=1}^{n} \delta_i DF(T_i)$.  

$F = \frac{DF(0) - DF(T)}{\sum_{i=1}^{n} \delta_i DF(T_i)} $

- We calculate the value of a cap or floor using normal formula.

$PV_\text{CAP} = N * LVL * C_i$. -------- / -------- $PV_\text{FLOOR} = N * LVL * P_i$.    

*where*:   
$N$: the nominal of the cap/floor.  
$C_i$: the value of the call option.  
$P_i$: the value of the put option.        
$LVL = \sum_{i=1}^{n} \delta_i DF(T_i)$  : the level.     

- We calculate the discounted value of the underlying swap as $PV_\text{Swap}  = N * LVL * (F - K)$, i.e. $PV_\text{Swap} = PV_\text{CAP} - PV_\text{FLOOR}$



In [38]:
"""
Vanilla instrument = caplet / floorlet or swaption
"""

class instrument :

    def __init__(self, call_put, exp_time, start_time, pay_times, year_fractions, strike, nominal = 100.):
        self.call_put = call_put
        self.exp_time = exp_time
        self.start_time = start_time
        self.pay_times = pay_times
        self.year_fractions = year_fractions
        self.strike = strike
        self.nominal = nominal

    @classmethod
    def swaption(cls, payer_receiver, expiry, tenor, strike, nominal = 100.):
        call_put = 'call' if payer_receiver == 'payer' else 'put'
        start_time = expiry + 2./365.
        pay_times = np.arange(start_time+1, expiry+tenor+1)
        year_fractions = np.full(len(pay_times), 1.)
        return cls(call_put, expiry, start_time, pay_times, year_fractions, strike, nominal)

    @classmethod
    def capletfloorlet(cls, caplet_floorlet, expiry, tenor, strike, nominal = 100.):
        call_put = 'call' if caplet_floorlet == 'caplet' else 'put'
        start_time = expiry + 2./365.
        pay_times = np.array([start_time + tenor])
        year_fractions = np.array([tenor*365./360.])
        return cls(call_put, expiry, start_time, pay_times, year_fractions, strike, nominal)

    # store DFs values to save computation time
    def set_market_data(self, df, normal_vol):
        self.df_start_time = df(self.start_time)
        self.df_pay_times = df(self.pay_times)
        self.normal_vol = normal_vol

    def forward(self):
        level = np.sum(self.df_pay_times * self.year_fractions)
        return (self.df_start_time - self.df_pay_times[-1]) / level
    
    # calculate market price of cap/floor
    def market_price(self):
        level = np.sum(self.df_pay_times * self.year_fractions)
        fwd = (self.df_start_time - self.df_pay_times[-1]) / level
        return self.nominal * level * norm_option(self.call_put, self.exp_time, self.strike, fwd, self.normal_vol)

    def pv_underlying(self): # pv of underlying swap or FRA
        level = np.sum(self.df_pay_times * self.year_fractions)
        fwd = (self.df_start_time - self.df_pay_times[-1]) / level
        cp = 1. if (self.call_put=='call') else -1.
        return cp * self.nominal * level * (fwd - self.strike)

    def print(self):
        print("----------------------------------------------------------------------------------")
        print("VANILLA INSTRUMENT | Expiry = {} | Tenor = {:.2f} | strike = {:.4f}% | nominal = {}".format(self.exp_time, self.pay_times[-1] - self.start_time, self.strike*100., self.nominal))
        if hasattr(self, 'normal_vol'):
            print("Forward = {:.4f}% | Normal vol = {:.4f}% | Price = {:.4f}".format(self.forward()*100, self.normal_vol*100, self.market_price()))
        print()

## Define and calibrate the Hull and White 1 factor model

**Hull and White 1 factor model** $ \Rightarrow dr_t = (\theta(t) - \lambda r_t) dt + \sigma(t) dW_t$

*where*:   
$\lambda$ : the mean reversion.  
$\sigma(t)$ : the instant volatility of the short rate $r_t$.   
$\theta(t)$ : the function fitting the zero coupon curve.  

The Hull and White 1 factor model is based on the dynamic of zero coupon with maturity T : $\frac{dB(t,T)}{B(t,T)} = r_t + \Gamma(t,T)dW_t$   
*where*: the instant volatility of the discount factor is $\Gamma(t,T) = \frac{\sigma(t)}{\lambda}(e^{-\lambda(T-t)}-1)$

To calibrate the model, we identify the variance $\phi$ of a series of swaptions with constant total maturity (i.e. maturity until the expiry date + tenor), at their expiry date and from their price.   
Based on the variance $\phi$ for each swaption's expiry date, we calculate the instant volatility $\sigma$ of the short rate $r_t$ for these same expiry dates, and interpolate it over the period needed to calculate the swaptions with HW1F model.   
Finally for each time *t* (defined in following part for a time-based grid) we calculate $\phi(t)$ from $\sigma(t)$.



***

### 1. Calculate the price of a caplet/floorlet or a swaption
We calculate the theoritical prices of a caplet/floorlet/swaption in HW1F for different $\phi$ defined by the solver *optimize.newton* to get the correct $\phi$ (see below *Calibrate the variance $\phi$*)
1. We calculate the volatility in HW1F model:   

$\sigma_\text{eff} = \sqrt{\frac{\phi(T_f)}{T_f}} \cdot \frac{e^{-\lambda (T_0 - T_f)} - e^{-\lambda (T_1 - T_f)}}{\lambda}$

2. For a one-time payment (i.e. a caplet of floorlet), we calculate the price: 

$\text{Price Caplet/Floorlet}(K) = N \cdot B(0,T_1) \cdot BS(\text{call/put}, T_f, K’, F, \sigma_\text{eff})$.   

*where*:   
$B(0,T_1)$ the discount factor at $T_1$ the payment date.  

$K' = 1 + \delta K $ the adjusted strike.   

$F = \frac{B(0,T_0)}{B(0,T_1)}$ the forward rate.  

$\Rightarrow$ $K'$ and $F$ from $L(t,T_1,T_2) = \frac{1}{\delta}(\frac{B(0,T_0)}{B(0,T_1)}-1)$ formula to calculate a forward rate

3. For a multiple-time payment (i.e. a swaption), we consider it as an option on a bonds portfolio so that we are able to calculate the sum of options on zero-coupon bonds. 
  
So we get the formula $\Rightarrow \text{swaption}(K) = \sum_{i=1}^n c_i \cdot \text{option on zero-coupon bonds}(T_i)$.    

From this formula we identify the value for which each payment of the option on zero-coupon bonds is equal to another one.  
   
To identify this value we use the solver *optimize.newton* with the function $f(x) = \sum_i c_i \, \frac{B(0,T_i)}{B(0,T_0)} \, e^{- \frac{1}{2} (\beta_i^2 - \beta_0^2)\phi - (\beta_i - \beta_0)x} = 1$    

*where*:   
the zero coupon discount factor $B(t,T)$ is a deterministic function $\Rightarrow $ $B(t,T) = B_{t,T}(X_t) = \frac{B(0,T)}{B(0,t)}e^{\frac{1}{2}\beta(t,T)^2\phi(t) - \beta(t,T)X_t}$
  
$c_i = \delta K$ : the coupon corresponding to the strike x the time period

$\beta(t;T) = \int_t^T e^{-\lambda(u-t)} \ du = \frac{1 - e^{-\lambda (t-T)}}{\lambda}$.  

$\beta_0 = \beta(t_0; t_f)$.    

$\beta_i = \beta(t_i; t_f)$.   

When the value $x_0$ from the solver is identified, we calculate the price :

$\text{Price swaption} = N \cdot B(0,T_0) \cdot \sum_i c_i \, BS(\text{option sur } B(0,T_i)) = N \cdot B(0,T_0) \cdot \sum_i c_i \, BS(\text{call/put}, T_f, K_i, F_i, \sigma_\text{eff})$

*where*:   
$i$ : the Cash-Flow for each option on bonds portfolio

$K_i = \frac{B(0,T_i)}{B(0,T_0)} \, e^{- \tfrac{1}{2}(\beta_i^2 - \beta_0^2)\phi - (\beta_i - \beta_0)x_0}$

$F_i = \frac{B(0,T_i)}{B(0,T_0)}$

***

### 2. Calibrate the variance $\phi$ 
1. For the cap/floor corresponding to each swaption originally defined when executing the project, we calculate their market prices (see formula above in the previous part). 

2. We are seeking $\phi$ such as :  $\text{defined cap/floor's market prices} = \text{theoritical price of caplet/floorlet/swaption in HW1F for a particular } \phi$.  

To get $\phi$ we use the solver *optimize.newton* with the function $f(\phi) = \text{theoritical price} - \text{market price}$.   

The theoritical price is calculated with formulas described above in *Calculate the price of a caplet/floorlet or a swaption*.  

In order to get the outcome of the solver converging, we use $\phi \approx \sigma_\text{n}^2 \times T$ where $\sigma_n$ is the normal volatility defined earlier, as the initial $\phi$ used in the method.   

The derivative $f'(\phi)$ used in the solver, is simply : $\frac{f(\phi + \epsilon) - f(\phi - \epsilon)}{2\epsilon}$

***

### 3. Calibrate the instant volatility $\sigma(t)$ of the short rate $r_t$
1. For each swaption defined when executing the project, we calibrate the variance $\phi$ with the method described above in *Calibrate the variance $\phi$*.   
So We get a cumulated variance $\Rightarrow  \phi(t_i) = \mathrm{Var}(X_{t_i}) $ with $t_i$ the maturity of the swaption.   
Consequently we get a series $\{ (t_i, \phi_i) \}_{i=1..N}$ from the series of swaptions at different maturities.  

2. We add $t_0 = 0$ with $\phi(0) = 0$, and $\sigma_{\min} = 0.0005$ to avoid erroneous volatilities.   

3. The evolution of the variance between $t_{i+1}$ and $t_i$ is $\phi(t_{i+1}) = e^{-2\lambda (t_{i+1}-t_i)} \, \phi(t_i) + \frac{\sigma_i^2}{2\lambda}\Big(1 - e^{-2\lambda (t_{i+1}-t_i)}\Big)$.     
So we calculate $\sigma_i = \sqrt{ \frac{(\phi(t_{i+1}) - e^{-2\lambda (t_{i+1}-t_i)} \phi(t_i)) \cdot 2\lambda}{1 - e^{-2\lambda (t_{i+1}-t_i)}} }$.  
If $\sigma_i$ is smaller than $\sigma_{\min}$ we consider $\sigma_{\min}$.

4. For the calculation of next $\sigma_i$, we start with the last used $\phi_i$.

5. We keep constant $\sigma(t)$ from the last maturity for any following maturity.  

6. As outcome we get $t = (0, t_1, t_2, \dots, t_N)$ and $\sigma_t = (\sigma_0, \sigma_1, \dots, \sigma_N)$, used in *Get $\phi_i$* to interpolate any $\phi_i$.

*** 

### 4. Get the the variance $\phi$
1. We interpolate $\sigma(t)$ over each swaption's expiry date, and assign $\sigma(t)$ to the time-based grid defined in next part *Calculate the price of a swaption using the Hull and White 1 factor model with Monte Carlo method.*

2. For each node *t* from the grid we get $\phi$ from the formula :   
$\phi(t_i) = \frac{\phi(t_{i-1}) e^{2\lambda t_{i-1}} + \frac{\sigma_{i-1}^2}{2\lambda}\left(e^{2\lambda t_i} - e^{2\lambda t_{i-1}}\right)}{e^{2\lambda t_i}}$

3. We interpolate $\phi$ over the grid's nodes *t*, so that we get $\phi(t)$ based on $\sigma(t)$ calculated for each initially defined swaption's expiry date.  






In [39]:
"""
Description & calibration of Hull & White 1 factor model
"""

# Hull & White 1 factor
class hw1f_model :

	def __init__(self, lam = None, t = None, sigma_t = None):
		self.lam = lam
		self.t = t
		self.sigma_t = sigma_t

	def print(self):
		print("------------------------------------")
		print("HW1F MODEL | Mean reversion = {:.2f}%\nVol sigma(t) (%) = ".format(self.lam*100.))
		with np.printoptions(precision=4, suppress=True):
			print(np.column_stack((self.t, self.sigma_t*100)))
		print()


	# HW1F price for calib instrument instr and for variance phi
	def price_from_phi (self, instr, phi):
		Tf  = instr.exp_time
		T0  = instr.start_time
		DF0 = instr.df_start_time
		DFs = instr.df_pay_times

		# caplet
		if (len(instr.pay_times) == 1) :
			T1  = instr.pay_times[0]
			delta = instr.year_fractions[0]
			vol = math.sqrt(phi / Tf) * (math.exp(-self.lam*(T0-Tf)) - math.exp(-self.lam*(T1-Tf))) / self.lam
			return instr.nominal * DFs[0] * bs_option(instr.call_put, Tf, 1. + delta * instr.strike, DF0 / DFs[0], vol)

		# swaption = bond option
		else :
			# bond option coefficients
			coefs = instr.year_fractions * instr.strike
			coefs[-1] += 1

			# precalcs
			beta0 = (1.- math.exp(-self.lam * (T0-Tf)) ) / self.lam
			betas = (1.- np.exp(-self.lam * (instr.pay_times-Tf)) ) / self.lam
			DFs_sur_DF0 = DFs / DF0
			betas_moins_beta0 = betas - beta0
			betas2_moins_beta02 = betas**2 - beta0**2

			# func to solve to find exercise frontier
			def f(x):
				return np.sum(coefs * DFs_sur_DF0 * np.exp(-0.5 * betas2_moins_beta02 * phi - betas_moins_beta0 * x)) - 1

			def fprime(x):
				return np.sum(- coefs * DFs_sur_DF0 * betas_moins_beta0 * np.exp(-0.5 * betas2_moins_beta02 * phi - betas_moins_beta0 * x))

			# DL de l'exponentielle pour trouver un bon guess
			guess_x = (-1 + np.sum(coefs * DFs_sur_DF0 * (1 - 0.5 * betas2_moins_beta02 * phi))) / np.sum(coefs * DFs_sur_DF0 * betas_moins_beta0)

			x0 = optimize.newton(f, guess_x, fprime)

			strikes = DFs_sur_DF0 * np.exp(-0.5 * betas2_moins_beta02 * phi - betas_moins_beta0 * x0)
			vols = math.sqrt(phi / Tf) * (math.exp(-self.lam*(T0-Tf)) - np.exp(-self.lam*(instr.pay_times-Tf))) / self.lam
			call_put = 'put' if (instr.call_put == 'call') else 'call'
			return instr.nominal * DF0 * np.sum(coefs * bs_option_np(call_put, Tf, strikes, DFs_sur_DF0, vols))


	# find variance phi for a given calibration instrument
	def calibrate_phi(self, instr):

		market_price = instr.market_price()

		def f(phi):
			return self.price_from_phi(instr, phi) - market_price

		def fprime(phi):
			eps = 1e-06
			return (f(phi+eps) - f(phi-eps)) / (2.*eps)

		guess_phi = instr.normal_vol**2 * instr.exp_time # guess déterminé en supposant que vol r_t = vol taux swap
		return optimize.newton(f, guess_phi, fprime)


	# calibrate sigma(t) term structure on a set of calibration instruments
	def calibrate_sigma (self, calib_set):

		phi = []
		for instr in calib_set:
			phi.append(self.calibrate_phi(instr))

		t = np.empty(len(calib_set)+1)
		sigma_t = np.empty(len(calib_set)+1)

		t[0] = 0
		last_phi = 0.
		sigma_min = 0.0005

		for i in range(len(calib_set)):

			t[i+1] = calib_set[i].exp_time
			exp = math.exp(-2.*self.lam*(t[i+1]-t[i]))
			first_term = exp * last_phi

			# var squeeze
			if (phi[i] < first_term):
				sigma_t[i] = sigma_min
				phi[i] = first_term + sigma_min**2 * (1.-exp) / (2.*self.lam) # update phi accordingly
				print("HW1F calibration : variance squeeze at step {}. Instrument [expiry = {} / tenor = {}] will be mispriced".format(i+1, calib_set[i].exp_time, calib_set[i].pay_times[-1] - calib_set[i].start_time))
			# everything ok
			else :
				sigma_t[i] = math.sqrt( (phi[i]-first_term) * 2.*self.lam / (1.-exp) )
				if (sigma_t[i]<sigma_min) :
					sigma_t[i] = sigma_min
					phi[i] = first_term + sigma_min**2 * (1.-exp) / (2.*self.lam)
					print("HW1F calibration : variance squeeze at step {}. Instrument [expiry = {} / tenor = {}] will be mispriced".format(i+1, calib_set[i].exp_time, calib_set[i].pay_times[-1] - calib_set[i].start_time))

			last_phi = phi[i]

		# extropolate after last exp date => same size for t & sigma_t
		sigma_t[-1] = sigma_t[-2]

		# store in class attributes
		self.t = t
		self.sigma_t = sigma_t
	

	# computes phi(t) = var(Xt) on time vector t (np.array[(n,)])
	def get_phi(self, t):
		new_t = np.append(self.t, t)
		new_t = np.delete(new_t, np.where(new_t>t[-1]))
		new_t = np.unique(np.sort(new_t))
		sigma = interp1d(self.t, self.sigma_t, kind='previous', fill_value="extrapolate")
		sigma_t = sigma(new_t)
		phi_new_t = np.empty(len(new_t))
		phi_new_t[0] = 0.
		exp_2_lam_t = np.exp(2.*self.lam*new_t)

		for i in range(1, len(new_t)):
			phi_new_t[i] = (phi_new_t[i-1] * exp_2_lam_t[i-1] + sigma_t[i-1]**2 * (exp_2_lam_t[i] - exp_2_lam_t[i-1])/(2.*self.lam)) / exp_2_lam_t[i]

		phi = interp1d(new_t, phi_new_t, kind='previous', fill_value="extrapolate")

		return phi(t)
	

	# price of vanilla instrument (slow, just for test & display purpose, not used in calibration)
	def price(self, instr):
		t = np.array([instr.exp_time])
		phi = self.get_phi(t).item()
		return self.price_from_phi(instr, phi)


## Calculate the price of a swaption using the Hull and White 1 factor model with Monte Carlo method.

### 1. Construct a grid with parameters.
We construct a time-based grid containing the swaptions' expiry dates.

*** 

### 2. Assign parameters to the grid's time-points.

1. We interpolate the instant volatility $\sigma(t)$ of the short rate $r_t$ over the time-based grid, using the previously calculated $\sigma(t)$ for each swaption's expiry date.    
2. For each node *t* (i.e. each swaption's expiry date and time-step defined through *dt*) in the grid we identify the corresponding $\sigma(t)$ and then $\phi(t)$ through *Get the variance $\phi$* defined in previous part.
3. We define discount factors to each node *t*.

*** 

### 3. Simulate the stochastic process $X$ in Monte Carlo with its discount factors curve.  
1. We generate a standard brownian movement as Gaussian variable with $Z \sim \mathcal{N}(0, 1)$.      

Then we iterate the process : $X_{t_{i+1}} = \frac{ X_{t_i} e^{\lambda t_i} + \int_{t_i}^{t_{i+1}} e^{\lambda u}\phi(u)\,du + \sigma(t_i)\sqrt{\frac{e^{2\lambda t_{i+1}}-e^{2\lambda t_i}}{2\lambda}} \, Z }{e^{\lambda t_{i+1}}}$ *where* $\sigma(t_i) \cdot \sqrt{ \frac{ e^{2\lambda t_{i+1}} - e^{2\lambda t_i} }{2\lambda e^{\lambda t_{i+1}} } } Z$ is the brownian of the process.  

The drift of the process directly comes from the Hull and White model $dX_t = -\lambda X_t \, dt + \sigma(t) \, dW_t - d\phi(t)$  *where* $\phi(t)$ the deterministic function fitting the rate curve $r_t = X_t + \phi(t)$.  
*N.B.*: So we don't use Euler to iterate the process here.

***

2. We generate the discount factors curve of the stochastic process $X$ : $B_{0,t_{i+1}}(X_t) = B_{0,t_i}(X_t) + e^{-\int_{t_i}^{t_{i+1}} \phi(u) du} + \exp\!\Big(-\int_{t_i}^{t_{i+1}} X_u \, du \Big)$.  

*where*   
- $B(0,t_i)$ the discount factor corresponding to the cumulated actualization until $t_i$.  
- $e^{-\int_{t_i}^{t_{i+1}} \phi(u) du} = \frac{B(0, t_{i+1})}{B(0, t_i)}$ the actualization between $t_{i+1}$ and $t_i$, given by the zero-coupon curve.    
- $\exp\!\Big(-\int_{t_i}^{t_{i+1}} X_u \, du \Big)$ the stochastic correction from $X_i$, with as approximation $\int_{t_i}^{t_{i+1}} X_u du \approx \frac{X_{i+1} + X_i}{2} \Delta t$.  

As a reminder $B(0, t) = \exp\left(-\int_0^t r_u \, du\right)$ *where* $r_u = \phi(u) + X_u$, and $\phi(u)$ taken into account in the zero-coupon curve. 

***

### 4. Reconstruct the discount factors curve in HW1F model.
We calculate the discount factors in HW1F model using the formula previously used in calculating the theoritical price of the defined swaptions (when calibrating the model): 

$B(0, T_1,T_2) = \frac{B(0,T_1)}{B(0,T_2)} \, e^{- \frac{1}{2} \beta^2 \phi - \beta X_i}$    

*where*:   
$\beta(t;T) = \int_t^T e^{-\lambda(u-t)} \ du = \frac{1 - e^{-\lambda (t-T)}}{\lambda}$.  

*** 

### 5. Calculate the payoff of the swaptions.
1. We calculate the discount factor of the start date $B(0,T_f,T_s)$ and the discount factors of the payment dates $B(0,T_f,T_{p_i})$.  

2. We calculate the level $LVL = \sum_i \delta B(0,T_f,T_{p_{i}})$.  

3. We calculate the swap rate $S = \frac{B(0,T_f,T_s) - B(0,T_f,T_{p_n})}{LVL}$ at maturity $T_{p_n}$ of the swaption. 

4. We calculate the payoff $e^{B_{0,t_{i}}(X_t)} \cdot N \cdot LVL \cdot \max(\pm(S - K), 0)$

*where*:   
- $e^{B_{0,t_{i}}(X_t)}$ the discount factor in the stochastic process $X$ at maturity. 

N.B. If the swaption is a payer swaption (i.e. a call on the swap rate) then $cp = 1$. If the swaption is a receiver swaption (i.e. a put on the swap rate) then $cp = -1$.

***

### 6. Calculate the price of the swaptions.

We calculate the price as the mean of the set of values calculated with Monte Carlo method, with its interval of confidence based on the standard deviation of the set of values.   
We compare the HW1F price to the price calculated with Black formula :
- We validate that the Black formula's result belongs to the interval of confidence calculated with Monte Carlo.
- We calculate the ratio of the difference between both methodologies by Black formula's result.




In [40]:
"""
Hull & White 1 factor Monte Carlo
"""

class hw1f_mc :

	def __init__(self, df, hw, Nsimul, dt, event_t):

		self.df_yc = df
		self.hw = hw
		self.Nsimul = Nsimul

		# MC time steps
		t_max = event_t[-1]
		mc_t = np.arange(0, t_max + dt, dt)

		# merge mc + event + hw times
		t = np.concatenate((mc_t, event_t, hw.t))
		t = np.delete(t, np.where(t>t_max)) # remove times larger that monte carlo t_max
		t = np.unique(np.sort(t))

		self.t = t # t as attribute for get_index method

		# some display...
		print("HW1F Monte Carlo\n=====================================")
		print("{:,} simulations | {} time steps".format(Nsimul, len(t)-1).replace(',', ' '))
		print("schedule = {}\n".format(t))

		# compute HW sigma(t) on MC schedule
		sigma = interp1d(hw.t, hw.sigma_t, kind='previous', fill_value="extrapolate")
		sigma_t = sigma(t)

		# compute phi(t) on MC schedule
		phi_t = hw.get_phi(t)
		self.phi_t = phi_t # phi_t as attribute for reconstruction formula

		# compute DFs on MC schedule
		df_t = df(t)

		# MC Nsteps
		Nsteps = len(t) - 1

		# generate random N(0,1)
		normal = np.random.normal(0, 1, (Nsteps, Nsimul))

		# state variable X array (MC time schedule x simulations)
		self.X = np.empty(shape=(Nsteps+1, Nsimul)) # X as attribute for reconstr. formula
		self.X[0,:] = 0.

		# terme d'actualisation exp(-int r_t dt)
		self.disc = np.empty(shape=(Nsteps+1, Nsimul)) # disc as attribute for discounted payoff calculations outside class
		self.disc[0,:] = 1

		# precalc exp(-lambda * ti)
		exp_lam_t = np.exp(hw.lam*t)

		# precalc of stdev factor of N(0,1) in X(t)
		stdev = sigma_t[:-1] * np.sqrt( (exp_lam_t[1:]**2 - exp_lam_t[:-1]**2) / (2.*hw.lam))

		# MC main loop : generate X and discount
		for i in range(Nsteps):
			int_exp_phi = 0.5 * (exp_lam_t[i]*phi_t[i] + exp_lam_t[i+1]*phi_t[i+1]) * (t[i+1] - t[i])
			self.X[i+1,:] = ( self.X[i,:] * exp_lam_t[i] + int_exp_phi + stdev[i] * normal[i,:] ) /  exp_lam_t[i+1]
			self.disc[i+1,:] = self.disc[i,:] * (df_t[i+1]/df_t[i]) * np.exp(-0.5 * (self.X[i+1,:] + self.X[i,:]) * (t[i+1] - t[i]))


	def get_index(self, time, tol=1e-10):
		idx = int(np.argmin(np.abs(self.t - time)))
		if abs(self.t[idx] - time) > tol:
			raise ValueError(f"HW1F Monte Carlo : get_index : time {time} not in schedule (closest={self.t[idx]})")
		return idx

	# reconstruction formula
	def df(self, time, maturity):
		i = self.get_index(time)
		beta = (1. - math.exp(-self.hw.lam*(maturity-time))) / self.hw.lam
		return (self.df_yc(maturity)/self.df_yc(time)) * np.exp(-0.5 * beta**2 * self.phi_t[i] - beta * self.X[i,:])

	# discount term
	def discount(self, time):
		i = self.get_index(time)
		return self.disc[i,:]


	# shortcut for payoff of swaption or caplet/floorlet
	def discounted_payoff(self, instr):
		df_start_time = self.df(instr.exp_time, instr.start_time)
		df_pay_times = np.empty(shape=(len(instr.pay_times), self.Nsimul))
		for i in range(len(instr.pay_times)):
			df_pay_times[i,:] = self.df(instr.exp_time, instr.pay_times[i])
		level = np.sum(df_pay_times * instr.year_fractions[:, np.newaxis], axis=0)
		swaprate = (df_start_time - df_pay_times[-1,:]) / level
		cp = 1. if instr.call_put == 'call' else -1.
		return self.discount(instr.exp_time) * instr.nominal * level * np.maximum(cp * (swaprate - instr.strike), 0.)


	# compute price, confidence interval & display
	def compute_price(self, discounted_payoff, option_name, closed_form_price = -1, display=True) :
		price = np.mean(discounted_payoff)
		if (display):
			Nsimul = len(discounted_payoff)
			stdev = np.std(discounted_payoff)
			IC_low = price - 1.96 * stdev / math.sqrt(Nsimul)
			IC_up = price + 1.96 * stdev / math.sqrt(Nsimul)
			in_out = ''
			print("------------------------------------------------------")
			if (closed_form_price == -1) :
				print("{} = {:.5f}".format(option_name, price) )
			else :
				print("{} = {:.5f} (closed-form price = {:.5f})".format(option_name, price, closed_form_price) )
				in_out = ' - IN' if (IC_low < closed_form_price < IC_up) else ' - OUT'
			print("Conf. interv. 95% = [{:.5f} ; {:.5f}]{}".format(IC_low, IC_up, in_out) )
			if (closed_form_price != -1):
				print("Rel. error : {:.4f}%".format(100.*(price- closed_form_price)/closed_form_price))
			print()

		return price

## Calculate the price of a swaption using the Hull and White 1 factor model with partial differential equation (PDE) method.

### 1. Construct a grid with parameters.
1. We construct a grid containing the swaptions' expiry dates.    
2. For each node *t* (i.e. each swaption's expiry date and time-step defined through *dt*) in the grid we identify the corresponding $\phi(t)$ through *Get the variance $\phi$* defined in previous part.
3. We define the axis space *x*, with $\phi(t)$ when *t* is max as maximum value and its opposite as minimum value.  

*** 

### 2. Calculate the price of the caplets using Crank-Nicholson method.
The method is defined in details in the project **pricer_local_vol_for_options** in part *Calculate the option prices with partial difference equation (PDE) method*.   
From the calculated payoff, we iterate backward through the 2D-grid.   
We use the library *solve_banded* to reverse the tridiagonal matrices and identify the vector $V_t$ at each iteration until $V_0$.

1. Define the coefficients and calculate the vector $V_t$ by iteration.
- The drift $\mu$ of the short rate $r_t$ is used to calculate the coefficients $\Rightarrow \mu = \Big( \tfrac{1}{2}\big(\phi(t_{i+1}) + \phi(t_i)\big) - \lambda X_t \Big)\, \Delta t$.   
Because $\int_t^{t+\Delta t} r_s ds = \int_t^{t+\Delta t} \big( \phi(s) + X_s \big)\, ds$, then $\int_{t_i}^{t_{i+1}} \phi(s)\,ds \;\approx\; \tfrac{1}{2}\big(\phi(t_{i+1})+\phi(t_i)\big)\, \Delta t$.  

- The variance is based on $\sigma(t)$ calibrated and interpolated earlier in HW1F model.   

- Regarding the short rate $r_t \Rightarrow \int_{t_i}^{t_{i+1}} r_s ds = \int_{t_i}^{t_{i+1}} \phi(s)\,ds + X_{t_i}\,\Delta t$ with $\int_{t_i}^{t_{i+1}} \phi(s)\,ds = \log \frac{B(0,t_i)}{B(0,t_{i+1})}$.   

2. Calculate the price of the caplets at each node.    
For each caplet with its expiry date, we calculate the corresponding payoffs depending on a range of $\phi(t)$, range defined previously.    

3. Calculate the price of the caplet at initiation.    
At initiation (i.e. at the 1st node), we consider the payoff calculated at the expiry date of the caplet and at *x = 0*, i.e. at the node in the middle of the axis space *x*.   






In [41]:
"""
Hull & White 1 factor finite differences
"""

class hw1f_pde :

	def __init__(self, df, hw, nx, nstdev, dt, event_t):

		self.df_yc = df
		self.hw = hw

		# PDE times
		t_max = event_t[-1] # must be = last maturity of priced product
		dt_min = t_max / 50. # 50 time steps minimum !
		dt = min(dt, dt_min)
		pde_t = np.arange(0, t_max + dt, dt)

		# merge pde + prod + hw times
		t = np.concatenate((pde_t, event_t, hw.t))
		t = np.delete(t, np.where(t>t_max)) # remove times larger than PDE t_max
		t = np.unique(np.sort(t))
		self.t = t # t as attribute for get_index method

		# compute phi(t) on PDE schedule (as attribute : needed for reconstruction formula)
		self.phi_t = self.hw.get_phi(self.t)
		
		# x space
		x_max = nstdev * math.sqrt(self.phi_t[-1])
		x_min = - x_max
		self.x = np.linspace(x_min, x_max, nx, endpoint=True)

	# compute & display PDE price
	def compute_price (self, payoff, name = 'PDE price', closed_form_price = -1, display = True) :
		t_start = time.time()

		# Crank Nicholson
		theta = 0.5

		# compute HW sigma(t) on PDE schedule
		sigma = interp1d(self.hw.t, self.hw.sigma_t, kind='previous', fill_value="extrapolate")
		sigma_t = sigma(self.t)

		# compute DFs on PDE schedule
		df_t = self.df_yc(self.t)

		# space & time dimensions
		nt = len(self.t)
		nx = len(self.x)
		dx = (self.x[-1] - self.x[0]) / (nx - 1.)

		# option value
		v = np.zeros((nt, nx)) # final price will be v[0, int((nx -1)/2)]

		# keep memory of times where payoff is updated (in fact, where self.df is called...)
		
		self.event_times = []
		# payoff at final date
		v[-1,:] = payoff(self.t[-1], v[-1,:])

		# backward loop from t = t_max - dt to t = 0
		for i in range(nt-2, -1, -1): # => i = nt-2 ... 0

			# precalcs
			dt = self.t[i+1] - self.t[i]
			var =  sigma_t[i]**2 * dt
			mu = (0.5*(self.phi_t[i+1]+self.phi_t[i]) - self.hw.lam*self.x) * dt
			r = self.x * dt + math.log(df_t[i]/df_t[i+1])

			alpha_u = (var + mu * dx) / (2.*dx**2)
			alpha_d = (var - mu * dx) / (2.*dx**2)
			alpha_c = np.full(nx, - var / dx**2) # alpha_c does not depend upon x => cst vector

			# limits
			alpha_u[0]  = mu[0] / dx
			alpha_c[0]  = -alpha_u[0]
			alpha_d[0]  = 0.
			alpha_u[-1] = 0.
			alpha_c[-1] = mu[-1] / dx
			alpha_d[-1] = -alpha_c[-1]

			# transition coefs
			p_ur = (1.-theta)*alpha_u
			p_dr = (1.-theta)*alpha_d
			p_cr = (1.-theta)*alpha_c + 1. - (1.-theta)*r
			p_ul = -theta*alpha_u
			p_dl = -theta*alpha_d
			p_cl = -theta*alpha_c + 1. + theta*r

			# compute right member
			right = np.empty(nx)
			right[0] = p_cr[0]*v[i+1, 0] + p_ur[0]*v[i+1, 1]
			right[1:nx-1] = p_dr[1:nx-1]*v[i+1,0:nx-2] + p_cr[1:nx-1]*v[i+1,1:nx-1] + p_ur[1:nx-1]*v[i+1,2:nx]
			right[-1] = p_cr[-1]*v[i+1,-1] + p_dr[-1]*v[i+1,-2]

			# backward step inverting tridiagonal matrix
			upper_band = np.insert(p_ul, 0, 0.)[:-1] # transform p_ul to call solve_banded (0 must be in first position)
			lower_band = np.append(p_dl[1:nx], 0)  # transform p_ul to call solve_banded (0 must be in last position)
			tridiag = np.array([upper_band, p_cl, lower_band])
			v[i,:] = solve_banded((1, 1), tridiag, right)

			# apply payoff to v
			v[i,:] = payoff(self.t[i], v[i,:])

		price = v[0, int((nx-1)/2)]

		if (display):
			print("------------------------------------------------------")
			if (closed_form_price == -1) :
				print("{} = {:.5f}".format(name, price) )
			else :
				print("{} = {:.5f} (closed-form price = {:.5f})".format(name, price, closed_form_price) )
			if (closed_form_price != -1):
				print("Rel. error : {:.4f}%".format(100.*(price-closed_form_price)/closed_form_price))
			if (self.event_times):
				print("PDE DF function called in payoff at event time(s) : {}".format(set(self.event_times)))
			else :
				print("Caution : no call to df function in payoff !")
			print("PDE computation time : {:.3f} seconds\n".format(time.time()-t_start))

		return price
	
    # find index of time in PDE times (time must belong to PDE schedule !!)
	def get_index(self, time, tol=1e-10):
		idx = int(np.argmin(np.abs(self.t - time)))
		if abs(self.t[idx] - time) > tol:
			raise ValueError(f"HW1F Monte Carlo : get_index : time {time} not in schedule (closest={self.t[idx]})")
		return idx
	
	#def get_index(self, time):
	#	if (time in self.t):
			#return int(np.where(np.isclose(self.t,time))[0])
		#else: exit("\nHW1F PDE get_index : time {} does not exist in PDE schedule !\n".format(time))

	# PDE df : returns DF values accross x (shape=(nx,))
	def df(self, t, T):
		i = self.get_index(t)
		self.event_times.append(t)
		beta = (1.-math.exp(-self.hw.lam*(T-t))) / self.hw.lam
		return (self.df_yc(T)/self.df_yc(t)) * np.exp(-0.5*beta**2*self.phi_t[i] - beta*self.x)

	# PDE df : version for a T vector => returns a (nx, len(T)) array
	def df_array(self, t, T):
		i = self.get_index(t)
		self.event_times.append(t)
		beta = (1.-np.exp(-self.hw.lam*(np.transpose(T)-t))) / self.hw.lam
		return (self.df_yc(T)/self.df_yc(t)) * np.exp(-0.5*beta**2*self.phi_t[i] - beta[np.newaxis,:]*self.x[:,np.newaxis])

	# pv of underlying swap / fra of a swaption / caplet
	def pv_underlying(self, instr, t):
		df_start_time = self.df(t, instr.start_time)
		df_pay_times = self.df_array(t, instr.pay_times)
		level = np.sum(df_pay_times * np.transpose(instr.year_fractions)[np.newaxis,:], axis=1)
		swaprate = (df_start_time - df_pay_times[:,-1]) / level
		cp = 1. if (instr.call_put=='call') else -1.
		return cp * instr.nominal * level * (swaprate - instr.strike)

## Execution of the project / Part 1 - Monte Carlo method for swaptions

#### 1. Define a zero coupon curve.
#### 2. Define swaptions used for calibration.
In order to calibrate the Hull and White 1 factor model, we need to define the surface of the implied volatility (depending on expiry date and tenor).    
Because we are in HW1F, we only define a subset of this surface.  
We choose this subset with a diagonal of swaptions : expiry + tenor = total maturity (being constant).        

Therefore we define a total maturity (i.e. expiry + tenor).   
For each swaption with this total maturity (e.g. from 2y/13y to 14y/1y for total maturity being 15 years) we calculate their discounted factors corresponding to the start and payment/end dates.  

Consequently we have short swaptions such as swaption 2y/13y, with sensitivity to long volatilities of the curve.    
And we have long swaptions such as swaption 14y/1y, with sensitivity to short volatilities of the curve.   
We get enough data to calibrate the instant volatility $\sigma(t)$ of the short rate $r_t$.

For information, here we have hardcoded swaptions' normal volatilities, identified from the market.

#### 3. Calibrate the Hull and White 1 factor model.
We calibrate the Hull and White 1 factor model with a defined $\lambda$ through calibration of $\sigma(t)$, using the swaptions previously defined.  
We print $\sigma(t)$ over the maturity of the swaptions.

#### 4. Calculate the prices of swaptions with Monte Carlo method.
Prior to calculating the prices, we set the swaptions' expiry dates as a variable, and define the number of simulations and duration of each time-period dt.    
Then for each swaption, we calculate its price using the calibrated Hull and White 1 factor model. 
  
*Pay for attention:*  
 The calculated prices should be about the same as the prices observed in the market, as we use a HW1F model calibrated on these same swaptions and their implied volatilities from the market.  

#### 5. Validate the calibrated model and control mean-reversion $\lambda$.   
Finally we calculate the price of a swaption not in the diagonal of swaptions, so that we can evaluate our calibration and mean-reversion $\lambda$.    
We use a mid-curve swaption 5y/5y/5y (expiry/start/tenor), i.e. an option on a swap paying each year between the 11th and 15th year.      
This swaption depends on short and long rates (strong correlation between both), which depend themselves on $\lambda$. 

So with a correct calibrated $\lambda$, the calculated price of the swaption 5y/5y/5y should be close enough to its market price.   





In [42]:
"""
Calibration of a diagonal of swaptions
Repricing of swaptions with Monte Carlo

"""

t0  = time.time()

# Define a zero coupon curve
maturities = np.array([0, 1, 2, 5, 10, 15, 20, 30])
zc_rates = np.array([-0.55, -0.52, -0.51, -0.4,  -0.18,  0.02,  0.11,  0.09]) * 0.01
yc = zc_curve(maturities, zc_rates)

# Define swaptions used for calibration
total_mat = 15 # Total maturity = expiry + tenor (e.g. for total maturity = 15 then set from 2y/13y to 14y/1y)
calib_set = []
for expiry in range (2, total_mat) :
	tenor = total_mat - expiry
	swaption = instrument.swaption('payer', expiry, tenor, 0.0025, 100.) # strike = 0.25% and nominal = 100
	swaption.set_market_data(yc.df, 0.0065) # market data contains for each swaption the start date's df, payment date's df and normal volatility (hardcoded here)
	calib_set.append(swaption)

# create hw1f model & calibrate
mean_reversion = 0.015
hw = hw1f_model(mean_reversion)
hw.calibrate_sigma(calib_set)
hw.print()

t1  = time.time()

# product event times => based on calib set as we reprice the calib set swaptions
event_t = []
for swaption in calib_set :
	event_t.append(swaption.exp_time)

# MC params
Nsimul = 10**6
dt = 1.

# generate MC simulations
mc = hw1f_mc(yc.df, hw, Nsimul, dt, event_t)

# Reprice all swaptions in calibration set
for swaption in calib_set: # in calib_set[3:4] to price only 5y/10y
	mc.compute_price(mc.discounted_payoff(swaption), 'Swaption {}y/{}y'.format(swaption.exp_time, int(swaption.pay_times[-1]-swaption.start_time)), hw.price(swaption))
	print("Swaption market price = {:.4f}\n".format(swaption.market_price()))

# mid-curve option 5/5/5 => depends on mean reversion value
mid_curve_option = instrument('call', 5, 10, np.arange(11, 16), np.full(5, 1.), 0.005, 100)
mc.compute_price(mc.discounted_payoff(mid_curve_option), 'Mid curve option (mean rev = {}%)'.format(hw.lam*100))

t2 = time.time()

print("\nCalibration time = {:.2f} seconds".format(t1 - t0))
print("Monte Carlo time = {:.2f} seconds".format(t2- t1))
print("TOTAL = {:.2f} seconds\n".format(t2- t0))

------------------------------------
HW1F MODEL | Mean reversion = 1.50%
Vol sigma(t) (%) = 
[[ 0.      0.7333]
 [ 2.      0.7294]
 [ 3.      0.7271]
 [ 4.      0.7253]
 [ 5.      0.7234]
 [ 6.      0.7215]
 [ 7.      0.7198]
 [ 8.      0.7184]
 [ 9.      0.7174]
 [10.      0.7171]
 [11.      0.7172]
 [12.      0.7175]
 [13.      0.7178]
 [14.      0.7178]]

HW1F Monte Carlo
1 000 000 simulations | 14 time steps
schedule = [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]

------------------------------------------------------
Swaption 2y/13y = 3.91040 (closed-form price = 3.91644)
Conf. interv. 95% = [3.89884 ; 3.92197] - IN
Rel. error : -0.1540%

Swaption market price = 3.9164

------------------------------------------------------
Swaption 3y/12y = 4.84992 (closed-form price = 4.85507)
Conf. interv. 95% = [4.83650 ; 4.86335] - IN
Rel. error : -0.1059%

Swaption market price = 4.8551

------------------------------------------------------
Swaption 4y/11y = 5.43857 (closed

## Execution of the project / Part 2 - Monte Carlo method for caplets and floorlets

We calculate the prices of caplets similar to the prices of swaptions calculated in the part above.     
We use caplets of maturity from 1 year to 20 years paid once a year, and a strike of 0.15%. We consider a constant market volatility.

So we calibrate the model for each caplet, on a cap EURIBOR 12M with strike 0.15% and normal volatility of 0.75%.

In [43]:
"""
Calibration of HW1F to a strip of caplets (Cap 20 years Euribor 12m)
Repricing of caplets with Monte Carlo

"""

t0  = time.time()

# yield curve t = 0
maturities = np.array([0, 1, 2, 5, 10, 15, 20, 30])
zc_rates = np.array([-0.55, -0.52, -0.51, -0.4,  -0.18,  0.02,  0.11,  0.09]) * 0.01
yc = zc_curve(maturities, zc_rates)

# caplets
total_mat = 20
calib_set = []
for expiry in range (1, total_mat) :
	caplet = instrument.capletfloorlet('caplet', expiry, 1., 0.0015, 100.)
	caplet.set_market_data(yc.df, 0.0075)
	calib_set.append(caplet)

# create hw1f model & calibrate
mean_reversion = 0.015
hw = hw1f_model(mean_reversion)
hw.calibrate_sigma(calib_set)
hw.print()

t1  = time.time()

# product event times => based on calib set as we reprice the calib set swaptions
event_t = []
for caplet in calib_set :
	event_t.append(caplet.exp_time)

# MC params
Nsimul = 10**6
dt = 1.

# generate MC simulations
mc = hw1f_mc(yc.df, hw, Nsimul, dt, event_t)

# Reprice all caplets in calibration set
for caplet in calib_set: # in calib_set[3:4] to price only 5y/10y
	mc.compute_price(mc.discounted_payoff(caplet), 'Caplet {}y'.format(caplet.exp_time), hw.price(caplet))
	print("Caplet market price = {:.4f}\n".format(caplet.market_price()))

t2 = time.time()

print("\nCalibration time = {:.2f} seconds".format(t1 - t0))
print("Monte Carlo time = {:.2f} seconds".format(t2- t1))
print("TOTAL = {:.2f} seconds\n".format(t2- t0))

------------------------------------
HW1F MODEL | Mean reversion = 1.50%
Vol sigma(t) (%) = 
[[ 0.      0.7733]
 [ 1.      0.7842]
 [ 2.      0.7943]
 [ 3.      0.8045]
 [ 4.      0.8149]
 [ 5.      0.8252]
 [ 6.      0.8351]
 [ 7.      0.8449]
 [ 8.      0.8545]
 [ 9.      0.8639]
 [10.      0.8737]
 [11.      0.8838]
 [12.      0.8942]
 [13.      0.9051]
 [14.      0.9161]
 [15.      0.9265]
 [16.      0.9366]
 [17.      0.9465]
 [18.      0.9564]
 [19.      0.9564]]

HW1F Monte Carlo
1 000 000 simulations | 19 time steps
schedule = [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]

------------------------------------------------------
Caplet 1y = 0.08362 (closed-form price = 0.08362)
Conf. interv. 95% = [0.08317 ; 0.08407] - IN
Rel. error : -0.0036%

Caplet market price = 0.0836

------------------------------------------------------
Caplet 2y = 0.20077 (closed-form price = 0.20085)
Conf. interv. 95% = [0.19993 ; 0.20160] - IN
Rel. error : -0.0422%


## Execution of the project / Part 3 - Partial differential equations method for caplets and floorlets.

We calculate the prices of caplets with PDE method, similar to the prices of caplets and swaptions calculated in parts above with Monte Carlo method.     
We use caplets of maturity from 1 year to 30 years paid twice a year, and a strike of 0.1%. We consider a constant market volatility.

So we calibrate the model for each caplet, on a cap EURIBOR 6M with strike 0.1% and normal volatility of 0.65%.


In [44]:
"""
Calibration to a cap 30y (by each caplet)
Repricing of caplets with PDE

"""

t0 = time.time()

# yield curve t = 0
maturities = np.array([0, 1, 2, 5, 10, 15, 20, 30])
zc_rates = np.array([-0.55, -0.52, -0.51, -0.4,  -0.18,  0.02,  0.11,  0.09]) * 0.01
yc = zc_curve(maturities, zc_rates)

# calibration sur set caplet euribor 6m
total_mat = 30
strike = 0.001
tenor = 0.5 # Euribor 6M
calib_set = []
for expiry in np.arange(0.5, total_mat, tenor) :
	caplet = instrument.capletfloorlet('caplet', expiry, tenor, strike, 100.)
	caplet.set_market_data(yc.df, 0.0065)
	calib_set.append(caplet)

#create hw1f model & calibrate
mean_rev = 0.02
hw = hw1f_model(mean_rev)
hw.calibrate_sigma(calib_set)
hw.print()
print("Calibration time = {:.3f} seconds\n\n".format(time.time() - t0))

# PDE params
nx = 601
nstdev = 5
dt = 0.05

# PDE price all caplets
for caplet in calib_set:
	def caplet_payoff(t, v_t):
		if (math.isclose(t, caplet.exp_time)):
			v_t = np.maximum(pde.pv_underlying(caplet, t), 0.)
		return v_t

	caplet.print()
	hw_price = hw.price(caplet)

	pde = hw1f_pde(yc.df, hw, nx, nstdev, dt, event_t = [caplet.exp_time])
	price = pde.compute_price(caplet_payoff, 'Caplet', hw_price)
	print("Market price : {:.5f}\n".format(caplet.market_price()))	

------------------------------------
HW1F MODEL | Mean reversion = 2.00%
Vol sigma(t) (%) = 
[[ 0.      0.6664]
 [ 0.5     0.6729]
 [ 1.      0.6794]
 [ 1.5     0.6857]
 [ 2.      0.6918]
 [ 2.5     0.6979]
 [ 3.      0.704 ]
 [ 3.5     0.71  ]
 [ 4.      0.716 ]
 [ 4.5     0.7221]
 [ 5.      0.728 ]
 [ 5.5     0.7339]
 [ 6.      0.7397]
 [ 6.5     0.7455]
 [ 7.      0.7512]
 [ 7.5     0.7569]
 [ 8.      0.7625]
 [ 8.5     0.768 ]
 [ 9.      0.7735]
 [ 9.5     0.779 ]
 [10.      0.7845]
 [10.5     0.79  ]
 [11.      0.7956]
 [11.5     0.8011]
 [12.      0.8066]
 [12.5     0.8122]
 [13.      0.8178]
 [13.5     0.8233]
 [14.      0.829 ]
 [14.5     0.8345]
 [15.      0.84  ]
 [15.5     0.8453]
 [16.      0.8506]
 [16.5     0.8559]
 [17.      0.8611]
 [17.5     0.8663]
 [18.      0.8715]
 [18.5     0.8766]
 [19.      0.8817]
 [19.5     0.8868]
 [20.      0.8918]
 [20.5     0.8968]
 [21.      0.9018]
 [21.5     0.9067]
 [22.      0.9115]
 [22.5     0.9163]
 [23.      0.9211]
 [23.5     0.9